In [1]:
%load_ext autoreload
%autoreload 2

In [59]:
import os, shutil, pickle, shelve
from Bio import SeqIO
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import load_model, Model
from keras.utils import plot_model
from keras.models import Model, Sequential
from keras.models import Sequential
from keras.layers import Dense, Dropout, MaxPooling1D, Flatten, Conv1D, LSTM, CuDNNLSTM, Bidirectional, TimeDistributed
from keras.optimizers import RMSprop, Adam
from keras.initializers import glorot_normal
from utilities import sampling, one_hot_encoding, curtail, get_training_data, load_data, data_split, dianostic_plots, pad_for_detector
from utilities import get_char_list, get_activated_subseq, get_freqs, get_candidates, get_motif
import keras
import utilities

In [4]:
# Subset sampling: save to me_samples; "me" for "mutually exclusive"
output_folder_path = "../../../../temp/buffers/me_samples"

data_dir = "/home/ubuntu/group_volume/team_neural_network/data/input/3.24_species_only"

len(os.listdir(data_dir)) # total number of regions

3543

### Start creating 5 folds of training data

In [18]:
L = len(os.listdir(data_dir))
k = 5
r = 1/(k + 1)
partition = [int(n*r*L) for n in range(0, k)] + [L]

In [19]:
partition

[0, 590, 1181, 1771, 2362, 3543]

In [22]:
folds = [np.arange(partition[i], partition[i+1]) for i in range(k)]

In [29]:
all_data_lst = np.array(os.listdir(data_dir))
fold_root_dir = '/home/ubuntu/data/temp/folds'
for i in range(k):
    fold_files = all_data_lst[folds[i]]
    fold_dir = os.path.join(fold_root_dir, 'fold'+str(i+1))
    for file in fold_files:
        shutil.copy(os.path.join(data_dir, file),
                              fold_dir)
    print('copied training samples to {}'.format(fold_dir))

copied training samples to /home/ubuntu/data/temp/folds/fold1
copied training samples to /home/ubuntu/data/temp/folds/fold2
copied training samples to /home/ubuntu/data/temp/folds/fold3
copied training samples to /home/ubuntu/data/temp/folds/fold4
copied training samples to /home/ubuntu/data/temp/folds/fold5


In [43]:
k_fold_data = []
i = 0
fold_output_path = '/home/ubuntu/data/temp/buffers/folds/'
for i in range(k):
    one_fold_output_path = os.path.join(fold_output_path, 'fold'+str(i+1))
    all_regions = one_hot_encoding(os.path.join(fold_root_dir, 'fold'+str(i+1)+'/'),
                                   os.path.join(one_fold_output_path, 'fold'+str(i+1)+'.data'))
    temp_x, temp_y = get_training_data(all_regions, one_fold_output_path,
                                       1000, 'fold'+str(i+1)+'_x.data', 'fold'+str(i+1)+'_y.data')
    k_fold_data.append((temp_x, temp_y))

save to /home/ubuntu/data/temp/buffers/folds/fold1/fold1.data
(14160, 1000, 4) (14160,)
save to /home/ubuntu/data/temp/buffers/folds/fold1/fold1_x.data
save to /home/ubuntu/data/temp/buffers/folds/fold1/fold1_y.data
save to /home/ubuntu/data/temp/buffers/folds/fold2/fold2.data
(14184, 1000, 4) (14184,)
save to /home/ubuntu/data/temp/buffers/folds/fold2/fold2_x.data
save to /home/ubuntu/data/temp/buffers/folds/fold2/fold2_y.data
save to /home/ubuntu/data/temp/buffers/folds/fold3/fold3.data
(14160, 1000, 4) (14160,)
save to /home/ubuntu/data/temp/buffers/folds/fold3/fold3_x.data
save to /home/ubuntu/data/temp/buffers/folds/fold3/fold3_y.data
save to /home/ubuntu/data/temp/buffers/folds/fold4/fold4.data
(14184, 1000, 4) (14184,)
save to /home/ubuntu/data/temp/buffers/folds/fold4/fold4_x.data
save to /home/ubuntu/data/temp/buffers/folds/fold4/fold4_y.data
save to /home/ubuntu/data/temp/buffers/folds/fold5/fold5.data
(28344, 1000, 4) (28344,)
save to /home/ubuntu/data/temp/buffers/folds/fol

### Finish creating 5 folds of data

## Start training model and extracting motifs on all 5 folds

In [44]:
data_x = pickle.load(open('/home/ubuntu/data/temp/buffers/folds/fold5/fold5_x.data', 'rb'))
data_y = pickle.load(open('/home/ubuntu/data/temp/buffers/folds/fold5/fold5_y.data', 'rb'))
train_x, train_y, val_x, val_y = data_split(data_x, data_y, seed = 157)
train_x, val_x = pad_for_detector(train_x, 10), pad_for_detector(val_x, 10)

28344 (22676, 1000, 4) (22676,) (5668, 1000, 4) (5668,)


In [62]:
n = 15 # number of filters
m = 10 # filter size
def get_hybrid(opt, num_filters, kernel_size):
    """  Return a hybrid network given a optimizer
    """
    model = Sequential()
    model.add(Conv1D(filters = num_filters, 
                     kernel_size = kernel_size, 
                     padding = 'valid',
                     activation = 'relu'))
    model.add(MaxPooling1D(pool_size = 5, strides = 5))
    model.add(Dropout(0.1))
    #model.add(Bidirectional(LSTM(20)))
    #model.add(Bidirectional(CuDNNLSTM(15, return_sequences=True)))
    model.add(Bidirectional(CuDNNLSTM(20)))
    model.add(Dropout(0.2))
    model.add(Dense(50))
    model.add(keras.layers.LeakyReLU(alpha=0.3))
    model.add(Dense(20))
    model.add(keras.layers.LeakyReLU(alpha=0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['acc'])
    return model


def train(model, train_x, train_y, val_data, config = {'epochs': 35, 'batch_size': 256}):
    """  Train model for a given config, training data, and validation data
    """
    epochs, batch_size = config['epochs'], config['batch_size']
    return model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, validation_data=val_data)

In [85]:
# Start training
# Set up some configurations
optimizers = {'adam': Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-2, amsgrad=False),
              'rmsprop': RMSprop(lr=1e-2, rho=0.9, epsilon=None, decay=1e-2)}
config = {'epochs': 100, 'batch_size': 562}
opt = optimizers['rmsprop']

In [86]:
save_model_path = '/home/ubuntu/data/team_neural_network/code/models'
k_fold_history = []
for i in range(1, k+1):
    print("************************************")
    print("Training on fold {}".format(i))
    
    data_x_path = '/home/ubuntu/data/temp/buffers/folds/fold{}/fold{}_x.data'.format(i, i)
    data_y_path = '/home/ubuntu/data/temp/buffers/folds/fold{}/fold{}_y.data'.format(i, i)
    # Data preparation
    data_x = pickle.load(open(data_x_path, 'rb'))
    data_y = pickle.load(open(data_y_path, 'rb'))
    train_x, train_y, val_x, val_y = data_split(data_x, data_y, seed = 157)
    train_x, val_x = pad_for_detector(train_x, m), pad_for_detector(val_x, 10)
    
    # Training
    model = get_hybrid(opt, num_filters = n, kernel_size = m)
    history = train(model, train_x, train_y, (val_x, val_y), config)
    
    # Saving models
    k_fold_history.append(history)
    model_name = 'hybrid_net_fold' + str(i) + '.h5'
    model.save(os.path.join(save_model_path, model_name))
    print("************************************")

************************************
Training on fold 1
14160 (11328, 1000, 4) (11328,) (2832, 1000, 4) (2832,)
Train on 11328 samples, validate on 2832 samples
Epoch 1/100
11328/11328 [==============================] - 9s 805us/step - loss: 0.7347 - acc: 0.5475 - val_loss: 0.6894 - val_acc: 0.5576
Epoch 2/100
11328/11328 [==============================] - 4s 352us/step - loss: 0.6854 - acc: 0.5640 - val_loss: 0.6841 - val_acc: 0.5576
Epoch 3/100
11328/11328 [==============================] - 4s 349us/step - loss: 0.6822 - acc: 0.5597 - val_loss: 0.6859 - val_acc: 0.5593
Epoch 4/100
11328/11328 [==============================] - 4s 337us/step - loss: 0.6759 - acc: 0.5620 - val_loss: 0.7783 - val_acc: 0.5576
Epoch 5/100
11328/11328 [==============================] - 4s 336us/step - loss: 0.6701 - acc: 0.5794 - val_loss: 0.7651 - val_acc: 0.4527
Epoch 6/100
11328/11328 [==============================] - 4s 335us/step - loss: 0.6467 - acc: 0.6018 - val_loss: 0.6311 - val_acc: 0.6243
Epoch

11328/11328 [==============================] - 3s 285us/step - loss: 0.2913 - acc: 0.8742 - val_loss: 0.3073 - val_acc: 0.8757
Epoch 59/100
11328/11328 [==============================] - 3s 288us/step - loss: 0.2902 - acc: 0.8750 - val_loss: 0.3142 - val_acc: 0.8676
Epoch 60/100
11328/11328 [==============================] - 3s 286us/step - loss: 0.2886 - acc: 0.8741 - val_loss: 0.3170 - val_acc: 0.8697
Epoch 61/100
11328/11328 [==============================] - 3s 290us/step - loss: 0.2872 - acc: 0.8781 - val_loss: 0.3024 - val_acc: 0.8799
Epoch 62/100
11328/11328 [==============================] - 3s 280us/step - loss: 0.2902 - acc: 0.8742 - val_loss: 0.3164 - val_acc: 0.8683
Epoch 63/100
11328/11328 [==============================] - 3s 281us/step - loss: 0.2834 - acc: 0.8774 - val_loss: 0.3107 - val_acc: 0.8718
Epoch 64/100
11328/11328 [==============================] - 3s 280us/step - loss: 0.2831 - acc: 0.8772 - val_loss: 0.2924 - val_acc: 0.8789
Epoch 65/100
11328/11328 [=======

11348/11348 [==============================] - 3s 283us/step - loss: 0.5371 - acc: 0.7302 - val_loss: 0.5200 - val_acc: 0.7493
Epoch 50/100
11348/11348 [==============================] - 3s 283us/step - loss: 0.5342 - acc: 0.7329 - val_loss: 0.4975 - val_acc: 0.7680
Epoch 51/100
11348/11348 [==============================] - 3s 283us/step - loss: 0.5271 - acc: 0.7360 - val_loss: 0.5291 - val_acc: 0.7412
Epoch 52/100
11348/11348 [==============================] - 3s 275us/step - loss: 0.5345 - acc: 0.7260 - val_loss: 0.5260 - val_acc: 0.7387
Epoch 53/100
11348/11348 [==============================] - 3s 274us/step - loss: 0.5238 - acc: 0.7393 - val_loss: 0.5022 - val_acc: 0.7592
Epoch 54/100
11348/11348 [==============================] - 3s 282us/step - loss: 0.5235 - acc: 0.7348 - val_loss: 0.4893 - val_acc: 0.7719
Epoch 55/100
11348/11348 [==============================] - 3s 279us/step - loss: 0.5199 - acc: 0.7370 - val_loss: 0.5008 - val_acc: 0.7581
Epoch 56/100
11348/11348 [=======

11328/11328 [==============================] - 3s 307us/step - loss: 0.6760 - acc: 0.5474 - val_loss: 0.6727 - val_acc: 0.5742
Epoch 7/100
11328/11328 [==============================] - 3s 307us/step - loss: 0.6744 - acc: 0.5530 - val_loss: 0.6715 - val_acc: 0.5664
Epoch 8/100
11328/11328 [==============================] - 3s 304us/step - loss: 0.6735 - acc: 0.5512 - val_loss: 0.6701 - val_acc: 0.5689
Epoch 9/100
11328/11328 [==============================] - 3s 306us/step - loss: 0.6710 - acc: 0.5580 - val_loss: 0.6748 - val_acc: 0.5279
Epoch 10/100
11328/11328 [==============================] - 4s 310us/step - loss: 0.6692 - acc: 0.5600 - val_loss: 0.6684 - val_acc: 0.5696
Epoch 11/100
11328/11328 [==============================] - 3s 306us/step - loss: 0.6690 - acc: 0.5662 - val_loss: 0.6667 - val_acc: 0.5802
Epoch 12/100
11328/11328 [==============================] - 3s 305us/step - loss: 0.6692 - acc: 0.5629 - val_loss: 0.6695 - val_acc: 0.5491
Epoch 13/100
11328/11328 [==========

11328/11328 [==============================] - 3s 272us/step - loss: 0.6112 - acc: 0.6582 - val_loss: 0.5995 - val_acc: 0.6840
Epoch 65/100
11328/11328 [==============================] - 3s 274us/step - loss: 0.6073 - acc: 0.6654 - val_loss: 0.5966 - val_acc: 0.6826
Epoch 66/100
11328/11328 [==============================] - 3s 276us/step - loss: 0.6080 - acc: 0.6622 - val_loss: 0.6071 - val_acc: 0.6638
Epoch 67/100
11328/11328 [==============================] - 3s 278us/step - loss: 0.6083 - acc: 0.6662 - val_loss: 0.6339 - val_acc: 0.6243
Epoch 68/100
11328/11328 [==============================] - 3s 275us/step - loss: 0.6090 - acc: 0.6618 - val_loss: 0.5961 - val_acc: 0.6790
Epoch 69/100
11328/11328 [==============================] - 3s 282us/step - loss: 0.6058 - acc: 0.6666 - val_loss: 0.5942 - val_acc: 0.6882
Epoch 70/100
11328/11328 [==============================] - 3s 285us/step - loss: 0.6024 - acc: 0.6688 - val_loss: 0.5932 - val_acc: 0.6893
Epoch 71/100
11328/11328 [=======

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



11348/11348 [==============================] - 3s 278us/step - loss: 0.5911 - acc: 0.6744 - val_loss: 0.5771 - val_acc: 0.6943
Epoch 89/100
11348/11348 [==============================] - 3s 284us/step - loss: 0.5888 - acc: 0.6759 - val_loss: 0.5693 - val_acc: 0.7003
Epoch 90/100
11348/11348 [==============================] - 3s 277us/step - loss: 0.5850 - acc: 0.6771 - val_loss: 0.5674 - val_acc: 0.7010
Epoch 91/100
11348/11348 [==============================] - 3s 280us/step - loss: 0.5865 - acc: 0.6827 - val_loss: 0.5758 - val_acc: 0.6932
Epoch 92/100
11348/11348 [==============================] - 3s 277us/step - loss: 0.5806 - acc: 0.6879 - val_loss: 0.5655 - val_acc: 0.7063
Epoch 93/100
11348/11348 [==============================] - 3s 286us/step - loss: 0.5816 - acc: 0.6819 - val_loss: 0.5695 - val_acc: 0.6901
Epoch 94/100
11348/11348 [==============================] - 3s 277us/step - loss: 0.5836 - acc: 0.6822 - val_loss: 0.5663 - val_acc: 0.6922
Epoch 95/100
11348/11348 [=======

22676/22676 [==============================] - 6s 277us/step - loss: 0.6598 - acc: 0.5649 - val_loss: 0.6622 - val_acc: 0.5707
Epoch 46/100
22676/22676 [==============================] - 6s 275us/step - loss: 0.6582 - acc: 0.5688 - val_loss: 0.6611 - val_acc: 0.5729
Epoch 47/100
22676/22676 [==============================] - 6s 274us/step - loss: 0.6590 - acc: 0.5661 - val_loss: 0.6610 - val_acc: 0.5716
Epoch 48/100
22676/22676 [==============================] - 6s 274us/step - loss: 0.6566 - acc: 0.5727 - val_loss: 0.6614 - val_acc: 0.5672
Epoch 49/100
22676/22676 [==============================] - 6s 275us/step - loss: 0.6574 - acc: 0.5700 - val_loss: 0.6622 - val_acc: 0.5699
Epoch 50/100
22676/22676 [==============================] - 6s 277us/step - loss: 0.6568 - acc: 0.5721 - val_loss: 0.6600 - val_acc: 0.5757
Epoch 51/100
22676/22676 [==============================] - 6s 280us/step - loss: 0.6565 - acc: 0.5726 - val_loss: 0.6585 - val_acc: 0.5707
Epoch 52/100
22676/22676 [=======

In [87]:
motifs_from_k_folds = {i:[] for i in range(1, k+1)}

for i in range(1, k+1):
    model_name = 'hybrid_net_fold' + str(i) + '.h5'
    model = load_model(os.path.join('../../models/', model_name))

    filters = model.layers[0].get_weights()[0]
    bias = model.layers[0].get_weights()[1]

    layer_name = model.layers[0].get_config()['name']
    intermediate_layer_model = Model(inputs=model.input,
                                     outputs=model.get_layer(layer_name).output)
    intermediate_output = intermediate_layer_model.predict(val_x)

    activated_subseq = get_activated_subseq(intermediate_output, val_x, m)
    for k in list(activated_subseq):
        char_list = get_char_list(activated_subseq[k])
        uniques, freqs = get_freqs(char_list)
        candidates = get_candidates(uniques, freqs, 0.45)
        motifs_from_k_folds[i].append(get_motif(candidates))

In [88]:
k = 5
for i in range(1, k+1):
    print("**************************")
    print("motifs extracted from fold {}\n".format(i))
    for seq in motifs_from_k_folds[i]:
        print(seq)
    print("**************************")

**************************
motifs extracted from fold 1

['TGTTTGAAGA', 'TGTTTGAAAA']
['GCAAATTGCG', 'GCAAATTGCT']
['AATATGTTCG', 'AATATGATCG', 'AATATGTGCG', 'AATATGAGCG']
['CCTGGTGGGC']
['AATGGAAGTT', 'AAGGGAAGTT', 'AATGTAAGTT', 'AAGGTAAGTT', 'AATGGCAGTT', 'AAGGGCAGTT', 'AATGTCAGTT', 'AAGGTCAGTT']
['TTATTCAGGT', 'CTATTCAGGT', 'TTATTCTGGT', 'CTATTCTGGT']
['TATTCAAATT', 'TATTCAAATC']
['AATACTACAA']
['TAAACAAATC', 'GAAACAAATC', 'TTAACAAATC', 'GTAACAAATC']
['CGGCACCAGC', 'CGGCGCCAGC']
['TCGATTTGAC']
['AGTGCATATC', 'AGTGCCTATC']
['CGTTGGTGCC', 'CGTCGGTGCC', 'CGTTGGTTCC', 'CGTCGGTTCC', 'CGTTGGTGCG', 'CGTCGGTGCG', 'CGTTGGTTCG', 'CGTCGGTTCG']
['CATAACTGAC', 'CATAACCGAC', 'CATAACTGAA', 'CATAACCGAA']
['TCATGTCATA', 'TCATCTCATA']
**************************
**************************
motifs extracted from fold 2

['GTATCCTACT', 'TTATCCTACT', 'GTATCATACT', 'TTATCATACT']
['CCAACAAAAG', 'CCCACAAAAG', 'CCAACAAATG', 'CCCACAAATG']
['CAAGCAGCAT', 'CAAGCAGCAC']
['CAACGCTCAA', 'CAACGCTCGA']
['GTAATTCCCG',